In [1]:
import teradatasql
import getpass
from getpass import *
import pandas as pd

query = 'SELECT * FROM COVID_REPORTING_V.COVID_PATIENT_GEO_ALL where "date" is not null AND LOOK_BACK_PERIOD=14;'
with teradatasql.connect(host='tdprod1', user=input('Username:'), password=getpass('Password:'), logmech='LDAP') as connect:
    df = pd.read_sql(query, connect)

Username: talasir
Password: ········


In [2]:
import pandas as pd
geo_f = df[0:0]
commanFilePath = r'C:\Users\TALASIR\Documents\GIS\to_be_processed\\'
FileName = 'COVID_PATIENT_GEO14'
df.to_csv(commanFilePath + FileName + '.csv', header=True, index=False)
for x,y in enumerate(df['AGE_GROUP'].unique().tolist()):
    geo_temp = pd.DataFrame(df[df['AGE_GROUP'].str.contains(y)]).reset_index()
    #print(geo_temp)
    filewithpathname = commanFilePath + FileName + '_' + y + '.csv'
    geo_temp.to_csv(filewithpathname, header=True, index=False)

In [2]:
from sklearn.cluster import DBSCAN
from shapely.geometry import MultiPoint, Polygon, MultiPolygon
import pandas as pd
import datetime
from dateutil import parser
import numpy as np
import geopandas as gpd
from dateutil import parser
import os
import re

folder = r'C:\Users\TALASIR\Documents\GIS\to_be_processed'
listOfFiles = list()
for subdir, dirs, files in os.walk(folder):
    for name in files:
        if "COVID_PATIENT_GEO21" in name:
            filepath = subdir + os.sep + name
            listOfFiles.append(filepath)
            
for i in range(len(listOfFiles)):
    datafile=listOfFiles[i]
    data=pd.read_csv(datafile)
    Look_back_period=data['LOOK_BACK_PERIOD'].unique().astype(str)
    Look_back_periodToStr = ' '.join([str(elem) for elem in Look_back_period]) 
    Age_Group=data['AGE_GROUP'].unique().astype(str)
    Age_GroupToStr = ' '.join([str(elem) for elem in Age_Group]) 
    data.Date=[parser.parse(t).date() for t in data.Date]
    start_date=None
    if start_date is None:
            start_date=np.min(data.Date.values)
            recheader=['id','centroid_x','centroid_y']
    end_date=datetime.datetime.now().date()
    datevals=[start_date+datetime.timedelta(days=i) for i in range((end_date-start_date).days+1)]
    recheader.extend(datevals)
    gpframe=gpd.GeoDataFrame(data,geometry=gpd.points_from_xy(data.lon, data.lat))
    gpframe.crs={'init': 'epsg:4326'}
    gpframe=gpframe.to_crs({'init': 'epsg:26917'})
    data['x']=[geom.x for geom in gpframe.geometry]
    data['y']=[geom.y for geom in gpframe.geometry]
    data=data.drop(columns=['geometry'])
    xy=data[['x','y']].values
    latlon=data[['lon','lat']].values
    dates=data['Date'].values
    minneighbors=10
    maxdistance=1000
    clusterData=pd.DataFrame(columns=recheader)
    clustering = DBSCAN(eps=maxdistance, min_samples=minneighbors,algorithm='kd_tree').fit(xy)
    labels = clustering.labels_
    total_clusters = len(set(labels))
    k=0
    geometries=[]
    geometries_line=[]
    ids=[]
    for i in range(total_clusters):
            ltln=latlon[labels==i]
            date=dates[labels==i]
            prjpoints=xy[labels==i]
            if len(ltln)!=0 and len(ltln)>=minneighbors:
                tdframe=pd.DataFrame(columns=['date'])
                tdframe.date=datevals
                newdframe=pd.DataFrame(columns=['date','counts'])
                #aggregate counts based on date
                unique, counts = np.unique(date, return_counts=True)
                newdframe.date=unique
                newdframe.counts=counts
                tdframe=tdframe.merge(newdframe,on='date',how='outer')
                tdframe=tdframe.fillna(0)
                cnts=np.cumsum(tdframe.counts.values)
                cnts[cnts<minneighbors]=0
                tdframe.counts=cnts
                #for generating centroids
                geom=MultiPoint(ltln)
                dat=[k,geom.centroid.x,geom.centroid.y]
                ids.append(k)
                geom_proj=MultiPoint(prjpoints)
                #convex hulls based on all members
                geom_hull=geom_proj.convex_hull
                if geom_hull.geom_type!='Polygon':
                    geom_hull=geom_hull.buffer(maxdistance)
                geom_line=geom_hull.boundary
                geometries.append(geom_hull)
                geometries_line.append(geom_line)
                dat.extend(tdframe.counts.values.tolist())
                clusterData.loc[len(clusterData)]=dat
                k+=1
                clusterData.to_csv(datafile.replace('.csv','')+"_centroids_"+str(maxdistance)+"_"+str(minneighbors)+'.csv',index=False)
                new_temp_df=pd.DataFrame(ids,columns=['id'])
                new_temp_df = new_temp_df.assign(look_back=Look_back_periodToStr)
                new_temp_df = new_temp_df.assign(age_group=Age_GroupToStr)
                #print(new_temp_df)
                gdf = gpd.GeoDataFrame(new_temp_df, geometry=geometries,crs={'init': 'epsg:26917'})
                gdf_line = gpd.GeoDataFrame(new_temp_df, geometry=geometries_line,crs={'init': 'epsg:26917'})
                gdf.to_file(datafile.replace('.csv','')+"_shapes_"+str(maxdistance)+"_"+str(minneighbors)+'.shp') 
                gdf_line.to_file(datafile.replace('.csv','')+"_shapes_linestring_"+str(maxdistance)+"_"+str(minneighbors)+'.shp')
    i+=1

C:\Users\TALASIR\AppData\Local\Continuum\anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\TALASIR\AppData\Local\Continuum\anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\TALASIR\AppData\Local\Continuum\anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred

In [3]:
#Merge shape files based on clustering type
import os
from pathlib import Path
import geopandas as gpd
import pandas as pd
folder = Path(r"C:\Users\TALASIR\Documents\GIS\output")
file = os.listdir(folder)
path = [os.path.join(folder, i) for i in file if ".shp" in i]
gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in path], 
                        ignore_index=True), crs={'init': 'epsg:26917'})
gdf.drop_duplicates().to_file(folder / 'Neighborhood.shp')

['C:\\Users\\TALASIR\\Documents\\GIS\\output\\COVID_PATIENT_GEO21_shapes_1000_10.shp', 'C:\\Users\\TALASIR\\Documents\\GIS\\output\\COVID_PATIENT_GEO21_shapes_linestring_1000_10.shp']


In [5]:
#shape file to csv
import ogr,csv,sys
import os
command = "ogr2ogr -f CSV {} {} -lco GEOMETRY=AS_WKT".format(r"C:\Users\TALASIR\Documents\GIS\to_be_processed\COVID_PATIENT_GEO21_shapes_linestring_500_5.csv",r"C:\Users\TALASIR\Documents\GIS\to_be_processed\COVID_PATIENT_GEO21_shapes_linestring_500_5.shp")
os.system(command)

0